In [27]:
import gymnasium as gym
import random
from tqdm import tqdm
from PLUMP_ENV import CardDeck, PlumpPlayer, PlumpEnv
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Example usage:
num_players = 4
table = PlumpEnv(num_players)
table.deal_cards(num_cards = 8)
table.get_estimates()

In [3]:
for i, p in enumerate(table.players):
    print(f"Player {i}")
    for card in p.hand:
        print(f"{card['rank']}: {card['suit']}")
    print('-.-'*10)

Player 0
6: Hearts
10: Diamonds
5: Clubs
8: Spades
3: Spades
8: Clubs
Ace: Clubs
3: Clubs
-.--.--.--.--.--.--.--.--.--.-
Player 1
6: Clubs
3: Diamonds
4: Spades
Ace: Spades
2: Clubs
8: Diamonds
King: Spades
10: Hearts
-.--.--.--.--.--.--.--.--.--.-
Player 2
4: Hearts
7: Spades
3: Hearts
Jack: Spades
Queen: Spades
King: Clubs
9: Clubs
6: Spades
-.--.--.--.--.--.--.--.--.--.-
Player 3
Queen: Diamonds
2: Diamonds
Jack: Clubs
5: Spades
4: Clubs
King: Diamonds
10: Spades
8: Hearts
-.--.--.--.--.--.--.--.--.--.-


In [4]:
table.play_the_hands()

In [23]:
num_players = 4  # Change the number of players as needed
stick_strategies =['Royal Count'] *4
assert len(stick_strategies)== num_players, f"You have {len(stick_strategies)} strategies but only {num_players}!"
env = PlumpEnv(num_players, stick_strategies)

num_hands = 10
if num_hands < 1 or num_hands > 20:
    print("Invalid input. Please enter a value between 20 and 1.")
    exit()

go_down_then_up = True  
if go_down_then_up:
    range_values = range(num_hands, 0, -1)  # Count down
    range_values = list(range_values) + list(range(1, num_hands + 1))  # Count up
else:
    range_values = range(num_hands, 0, -1)  # Count down

for num_cards in range_values:
    print(num_cards)
    env.play_full_hand(num_cards=num_cards)

# Print the final results and protocol
print("Final Results:")
for player in env.players:
    print(f"Player {player.player_id}: Points={player.points}")

print("Protocol:")
for player_id, points_list in env.protocol.items():
    print(f"Player {player_id}: {points_list}")

10
9
8
7
6
5
4
3
2
1
1
2
3
4
5
6
7
8
9
10
Final Results:
Player 0: Points=114
Player 1: Points=84
Player 2: Points=69
Player 3: Points=93
Protocol:
Player 0: [13, 0, 12, 11, 11, 0, 12, 0, 0, 0, 0, 11, 0, 10, 0, 11, 0, 0, 11, 12]
Player 1: [0, 0, 0, 11, 10, 11, 0, 10, 11, 10, 0, 10, 0, 0, 11, 0, 0, 0, 0, 0]
Player 2: [0, 13, 0, 0, 0, 0, 10, 0, 0, 0, 10, 0, 11, 0, 12, 0, 0, 0, 13, 0]
Player 3: [0, 0, 0, 0, 13, 12, 11, 0, 11, 11, 0, 11, 0, 0, 11, 13, 0, 0, 0, 0]


In [34]:
import numpy as np

num_players = 4  # Change the number of players as needed
num_simulations = 1000  # Number of Monte Carlo simulations
stick_strategies1 =['Royal Count'] *4
stick_strategies2 =['Random'] *4

def run_simulation(num_players, num_hands, stick_strategies, go_down_then_up):
    assert len(stick_strategies)== num_players, f"You have {len(stick_strategies)} strategies but only {num_players}!"

    env = PlumpEnv(num_players, stick_strategies)
    if num_hands < 1 or num_hands > 20:
        print("Invalid input. Please enter a value between 20 and 1.")
        exit()

    if go_down_then_up:
        range_values = range(num_hands, 0, -1)  # Count down
        range_values = list(range_values) + list(range(1, num_hands + 1))  # Count up
    else:
        range_values = range(num_hands, 0, -1)  # Count down

    for num_cards in range_values:
        env.play_full_hand(num_cards=num_cards)

    return [player.points for player in env.players]

# Run Monte Carlo simulations
go_down_then_up = True
results1 = {}
results2 = {}
for n in tqdm((range(10, 0, -1)), unit= 'Hand size'):
    result1 = []
    result2 = []
    for _ in range(num_simulations):
        points1 = run_simulation(num_players=num_players, num_hands=n, stick_strategies=stick_strategies1, go_down_then_up=go_down_then_up)
        points2 = run_simulation(num_players=num_players, num_hands=n, stick_strategies=stick_strategies2, go_down_then_up=go_down_then_up)
        result1.append(np.mean(points1))
        result2.append(np.mean(points2))
    average_points1 = np.mean(result1)
    average_points2 = np.mean(result2)
    results1[n] = average_points1
    results2[n] = average_points2

print(f"Strategy {stick_strategies1}")
for n, p in results1.items():
    print(f"Given starting num_hands = {n}, the expected value is = {p}")
print('-.-'*20)
print(f"Strategy {stick_strategies2}")
for n, p in results2.items():
    print(f"Given starting num_hands = {n}, the expected value is = {p}")


100%|██████████| 10/10 [01:50<00:00, 11.02s/# of cards being simulated]

Strategy ['Royal Count', 'Royal Count', 'Royal Count', 'Royal Count']
Given starting num_hands = 10, the expected value is = 74.7805
Given starting num_hands = 9, the expected value is = 69.87125
Given starting num_hands = 8, the expected value is = 63.6295
Given starting num_hands = 7, the expected value is = 57.42625
Given starting num_hands = 6, the expected value is = 50.72025
Given starting num_hands = 5, the expected value is = 44.272
Given starting num_hands = 4, the expected value is = 37.16925
Given starting num_hands = 3, the expected value is = 29.878
Given starting num_hands = 2, the expected value is = 22.82925
Given starting num_hands = 1, the expected value is = 13.87575
-.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.-
Strategy ['Random', 'Random', 'Random', 'Random']
Given starting num_hands = 10, the expected value is = 42.911
Given starting num_hands = 9, the expected value is = 40.016
Given starting num_hands = 8, the expected value is = 38.30125
Given st

In [129]:
import numpy as np

# Define the range of num_hands and num_players
num_simulations = 10000  # Number of Monte Carlo simulations
num_hands_range = range(10, 0, -1)
num_players_range = range(5, 1, -1)  # From 5 to 2

# Run Monte Carlo simulations for different combinations of num_hands and num_players
results = {}
for num_hands in num_hands_range:
    results[num_hands] = {}
    for num_players in num_players_range:
        for _ in range(num_simulations):
            result = run_simulation(num_hands, num_players, go_down_then_up)
            results[num_hands][num_players] = np.mean(result)

In [131]:
for num_hands in num_hands_range:
    for num_players in num_players_range:
        expected_value = results[num_hands][num_players]
        print(f"Num_Hands={num_hands}\tNum_Players={num_players}\tExpected Value={expected_value:.3f}")
    print('-.-'*20)

Num_Hands=10	Num_Players=5	Expected Value=30.500
Num_Hands=10	Num_Players=4	Expected Value=24.200
Num_Hands=10	Num_Players=3	Expected Value=21.300
Num_Hands=10	Num_Players=2	Expected Value=11.100
-.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.-
Num_Hands=9	Num_Players=5	Expected Value=24.889
Num_Hands=9	Num_Players=4	Expected Value=31.556
Num_Hands=9	Num_Players=3	Expected Value=19.222
Num_Hands=9	Num_Players=2	Expected Value=17.000
-.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.-
Num_Hands=8	Num_Players=5	Expected Value=29.625
Num_Hands=8	Num_Players=4	Expected Value=28.500
Num_Hands=8	Num_Players=3	Expected Value=26.625
Num_Hands=8	Num_Players=2	Expected Value=17.875
-.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.-
Num_Hands=7	Num_Players=5	Expected Value=35.571
Num_Hands=7	Num_Players=4	Expected Value=28.000
Num_Hands=7	Num_Players=3	Expected Value=20.429
Num_Hands=7	Num_Players=2	Expected Value=13.143
-.--.--.--.--.--.--.--.--.--.--.--.--.--.--.-